In [5]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import openai
import base64
import requests
from io import BytesIO
import json

# Global variable to store the conversation history
conversation_history = []

# Function to encode the image to base64
def encode_image(image_file):
    return base64.b64encode(image_file.read()).decode('utf-8')

def send_message(b=None):
    global api_key, conversation_history
    message = message_input.value
    temperature = temperature_control.value
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    
    # Construct the payload
    payload = {
        "model": "gpt-4-vision-preview",
        "messages": conversation_history,
        "max_tokens": 2000,
        "temperature": temperature
    }
    
    # Check if there is an image to send
    if image_upload.value:
        try:
            uploaded_file = image_upload.value[0]
            image_data = uploaded_file['content']
            base64_image = encode_image(BytesIO(image_data))
            user_message = {
                "role": "user",
                "content": [
                    {"type": "text", "text": message},
                    {"type": "image_url", "image_url": f"data:image/jpeg;base64,{base64_image}"}
                ]
            }
        except Exception as e:
            print(f"Error processing the uploaded file: {e}")
            return
    else:
        user_message = {
            "role": "user",
            "content": [{"type": "text", "text": message}]
        }
    
    # Append the user's message to the conversation_history
    conversation_history.append(user_message)
    
    # Send the message
    try:
        response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
        response.raise_for_status()
        response_data = response.json()
        if 'choices' in response_data and response_data['choices']:
            ai_message_content = response_data['choices'][0]['message']['content']
            ai_message = {
                "role": "assistant",
                "content": ai_message_content
            }
            # Append the AI's response to the conversation_history
            conversation_history.append(ai_message)
            
            # Display the user's message and image if uploaded
            user_message_widget = widgets.HBox([])
            user_message_text = widgets.HTML(value=f"<div style='word-wrap: break-word;'>You: {message}</div>")
            user_message_widget.children += (user_message_text,)
            
            if image_upload.value:
                # Display the image inline
                image_widget = widgets.Image(value=image_data, format='jpeg', width=300, height=300)
                user_message_widget.children += (image_widget,)
            
            messages_display.children += (user_message_widget,)
            
            # Display the AI's response
            ai_message_widget = widgets.HTML(value=f"<div style='word-wrap: break-word;'>GPT-4V: {ai_message_content}</div>")
            messages_display.children += (ai_message_widget,)
        else:
            print("Failed to send message. Response did not contain expected data.")
    except Exception as err:
        print(f"An error occurred: {err}")
    finally:
        # Clear the input
        message_input.value = ""
        image_upload.value = tuple()

# Function to clear the conversation
def clear_conversation(b=None):
    global conversation_history
    conversation_history.clear()
    messages_display.children = []

# Function to download the conversation history as HTML
def download_conversation_as_html(b=None):
    # Start the HTML document with a style that ensures images keep their aspect ratio
    html_document = """
    <!DOCTYPE html>
    <html>
    <head>
    <style>
    .user {color: blue;}
    .assistant {color: green;}
    img {max-width: 100%; height: auto;}
    </style>
    </head>
    <body>
    """
    
    # Convert conversation_history to HTML format
    for message in conversation_history:
        role = message['role']
        content = message['content']
        if role == 'user':
            # Extract the text and image from the user's message content
            user_text = ""
            user_image_b64 = ""
            for part in content:
                if part['type'] == 'text':
                    user_text += part['text'] + " "
                elif part['type'] == 'image_url':
                    # Assuming the image is already in base64 in the conversation history
                    user_image_b64 = part['image_url'].split(',')[1]
            html_document += f"<p class='user'>User: {user_text}</p>\n"
            if user_image_b64:
                html_document += f"<img src='data:image/jpeg;base64,{user_image_b64}'/>\n"
        elif role == 'assistant':
            # Assistant messages are assumed to be plain text
            html_document += f"<p class='assistant'>Assistant: {content}</p>\n"
    
    # End the HTML document
    html_document += "</body>\n</html>"

    # Encode the HTML document to base64
    b64 = base64.b64encode(html_document.encode()).decode()
    payload = f"data:text/html;base64,{b64}"
    downloader = widgets.HTML(
        value=f'<a download="conversation.html" href="{payload}" target="_blank">Download Conversation as HTML</a>'
    )
    display(downloader)

# Function to save the API key
def save_api_key(b):
    global api_key
    api_key = api_key_input.value
    with output:
        clear_output()
        print("API Key saved successfully.")

# API Key Configuration
api_key_input = widgets.Text(description='API Key:', placeholder='Enter your OpenAI API key here...')
api_key_button = widgets.Button(description="Save API Key")
api_key_button.on_click(save_api_key)

# Chat Interface
message_input = widgets.Textarea(description='Message:')
image_upload = widgets.FileUpload(description='Upload Image', accept='image/*', multiple=False)
send_button = widgets.Button(description='Send Message')
send_button.on_click(send_message)
clear_button = widgets.Button(description='Clear Conversation')
clear_button.on_click(clear_conversation)
download_button = widgets.Button(description='Download Conversation')
download_button.on_click(download_conversation_as_html)
messages_display = widgets.VBox([], layout={'overflow': 'auto'})
temperature_control = widgets.FloatSlider(value=0.7, min=0, max=1.0, step=0.01, description='Temperature:')

# Display and Interaction
output = widgets.Output()
api_key_box = widgets.VBox([api_key_input, api_key_button])
controls_box = widgets.VBox([message_input, image_upload, send_button, clear_button, download_button, temperature_control])
chat_box = widgets.VBox([controls_box, messages_display])

# Layout the widgets
display(api_key_box, chat_box, output)


Output()

HTML(value='<a download="conversation.html" href="data:text/html;base64,CiAgICA8IURPQ1RZUEUgaHRtbD4KICAgIDxodG…